In [0]:
from pyspark.sql.functions import col

In [0]:
%run "../includes/common_functions"

In [0]:
latest_update = spark.sql("""select * 
                          from cars_catalog.default.metadata_table
                          where table_name = 'dim_branch' 
                          """).collect()[0][1]
latest_update

---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
File <command-4430124732203713>, line 1
----> 1 latest_update = spark.sql("""select * 
      2                           from cars_catalog.default.metadata_table
      3                           where table_name = 'dim_date' 
      4                           """).collect()[0][1]
      5 latest_update

IndexError: list index out of range

In [0]:
dim_table = spark.read.format("delta") \
                        .load("abfss://gold@cardeprojectdl.dfs.core.windows.net/dim_branch") \
                        .filter(col("updated_at") == latest_update)

In [0]:
dim_table.display()

Branch_ID,BranchName,updated_at,dim_branch_key
BR9546,Premier Motors,2025-05-06T08:33:00.291Z,1837
BR9666,Puma Motors,2025-05-06T08:33:00.291Z,1838
XYZ9726,DataFam Motors,2025-05-06T08:33:00.291Z,1839
BR9726,Power Ranger Motors,2025-05-06T08:33:00.291Z,1840


In [0]:
sql_create_table = """
    CREATE TABLE IF NOT EXISTS CARSALES_PROJECT.DW.dim_branch (
        BranchName STRING,
        Branch_ID STRING,
        dim_branch_key NUMBER,
        updated_at TIMESTAMP
    )

"""

In [0]:
sql_merging = """
            MERGE INTO CARSALES_PROJECT.DW.DIM_BRANCH AS tar
            USING CARSALES_PROJECT.EXTERNAL_STAGE.STG_DIM_BRANCH AS src
            ON tar.dim_branch_key = src.dim_branch_key
            WHEN MATCHED THEN
                UPDATE SET tar.BranchName = src.BranchName,
                            tar.Branch_ID = src.Branch_ID,
                            tar.updated_at = src.updated_at
            WHEN NOT MATCHED THEN   
                INSERT (Branch_ID, BranchName, dim_branch_key, updated_at) VALUES (src.Branch_ID, src.BranchName, src.dim_branch_key, src.updated_at)
"""

In [0]:
write_to_snowflake(dim_table,"STG_DIM_BRANCH",sql_create_table,sql_merging)